In [1]:
import imp
import numpy as np
import pandas as pd
import openpyxl
import os 
import xlrd
# Setting Directory
os.chdir('C:/Users/TAM/Documents/Vu Quang Nguyen/VACP/')

C:\Users\TAM\AppData\Local\Temp\ipykernel_12944\2202898749.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
# NHAP - XUAT - TON  * GIA BAN * DANH MUC HANG HOA

data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5).rename(columns={
                                "Unnamed: 0":"ID",
                                "Unnamed: 1":"Tên vật tư",
                                "Số lượng":"Tổng đầu kỳ", 
                                "Giá trị":"Giá trị đầu kỳ",
                                "Số lượng.1":"Tổng nhập", 
                                "Giá trị.1":"Giá trị nhập",
                                "Số lượng.2":"Tổng xuất", 
                                "Giá trị.2":"Giá trị xuất",
                                "Số lượng.3":"Tổng tồn", 
                                "Giá trị.3":"Giá trị tồn" }).dropna(subset=['ID']).reset_index(drop=True)

data_muahang=pd.read_excel("../VACP/Python NCC Min/Sổ chi tiết mua hàng (2).xls",usecols="A,C,F,I:J",skipfooter=1,header=5).rename(columns={
                                "Unnamed: 0":"Ngày nhập",
                                "Unnamed: 2":"Nhà cung cấp",
                                "Unnamed: 5":"ID",
                                "Số lượng":"Tổng nhập", 
                                "Đơn giá":"Giá nhập"
                                })
                                
data_banhang=pd.read_excel("../VACP/Bao cao Ban hang Power BI/Sổ chi tiết bán hàng.xls",usecols="A,C:D,F,H,K:M",skipfooter=1,header=6).rename(columns={
                                "Unnamed: 0":"Ngày bán",
                                "Unnamed: 2":"ID Khách hàng",
                                "Unnamed: 3":"Hãng",
                                "Unnamed: 5":"ID",
                                "Unnamed: 7":"Tên hàng",
                                "Số lượng":"Tổng bán", 
                                "Đơn giá gốc":"Giá bán",
                                "Giá bán":"Giá bán giảm"
                                }).dropna(subset=['ID']).reset_index(drop=True)
                                
data_giaban=pd.read_excel("../VACP/Python NCC Min\Báo cáo giá bán, tồn kho.xls",usecols="C,F",skipfooter=1,header=4).rename(columns={"Mã số":"ID"}).dropna(
                subset=['ID']).reset_index(drop=True)

data_danhmuchanghoa=pd.read_excel("../VACP/Python NCC Min/Danh mục vật tư hàng hóa .xls",usecols="A,C:D,M",header=2).rename(
                            columns={
                                "Mã nhóm":"ID Nhóm hàng",
                                "Mã":"ID",
                                "Tên in":"Tên vật tư",
                                "Hãng sản xuất":"Hãng"
                                }).dropna(subset=['ID','ID Nhóm hàng']).reset_index(drop=True)



C:\Users\TAM\AppData\Local\Temp\ipykernel_12944\401360059.py:3: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5).rename(columns={


In [3]:
# Giá vốn
TransCost=0.05
data_nhapxuatton['Giá vốn'] =np.round(
                                         (1+TransCost)*np.where(data_nhapxuatton["Giá trị xuất"]>0,
                                        data_nhapxuatton["Giá trị xuất"]/data_nhapxuatton["Tổng xuất"],
                                            np.where(data_nhapxuatton["Giá trị tồn"]>0,
                                                data_nhapxuatton["Giá trị tồn"]/data_nhapxuatton["Tổng tồn"],0

                                            ))
                                    ,0)

In [4]:
#Mua hàng giá nhập nhỏ nhất
min_mua_hang=data_muahang.groupby(["ID"])["Giá nhập"].min()
data_muahang_min=data_muahang.merge(min_mua_hang,on="ID",how="left",suffixes=('', '_min'))
data_muahang_min=data_muahang_min[data_muahang_min["Giá nhập"]==data_muahang_min["Giá nhập_min"]
                        ].drop('Giá nhập_min',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True).rename(
                        columns={
                            "Ngày nhập":"Ngày nhập MIN",
                            "Nhà cung cấp":"Nhà cung cấp MIN",
                            "Tổng nhập":"Tổng nhập MIN",
                            "Giá nhập":"Giá nhập MIN"
                        })



In [5]:
#Mua hàng gần đây
max_date_muahang=data_muahang.groupby(["ID"])["Ngày nhập"].max()
data_muahang_latest=data_muahang.merge(max_date_muahang,on="ID",how="left",suffixes=('', '_max'))
data_muahang_latest=data_muahang_latest[data_muahang_latest["Ngày nhập"]==data_muahang_latest["Ngày nhập_max"]
                ].drop('Ngày nhập_max',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True).rename(
                        columns={
                            "Ngày nhập":"Ngày nhập gần đây",
                            "Nhà cung cấp":"Nhà cung cấp gần đây",
                            "Tổng nhập":"Tổng nhập gần đây",
                            "Giá nhập":"Giá nhập gần đây"
                        }
                    )
 

In [6]:
#Bán hàng gần đây
max_date_banhang=data_banhang.groupby(["ID"])["Ngày bán"].max()
data_banhang_latest=data_banhang.merge(max_date_banhang,on="ID",how="left",suffixes=('', '_max'))
data_banhang_latest=data_banhang_latest[data_banhang_latest["Ngày bán"]==data_banhang_latest["Ngày bán_max"]
                    ].drop('Ngày bán_max',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True).rename(
                        columns={
                            "Ngày bán":"Ngày bán gần đây",
                            "Giá bán giảm":"Giá bán gần đây"
                        }
                    )

 

In [7]:
data_banhang_latest

,Ngày bán gần đây,ID Khách hàng,Hãng,ID,Tên hàng,Tổng bán,Giá bán,Giá bán gần đây
0,2022-01-06,KIET-CHUBA,TOYOTA,16325-28010,Gioăng van hằng nhiệt,2,80000.0,80000.0
1,2022-01-06,HUYNHTANVT,HONDA,51606-SWA-J01,Giảm sóc trước CRV Lh,1,NaN,1100000.0
2,2022-02-11,KIET-CHUBA,TOYOTA,48530-80525,Giảm sóc sau ACV40 RH,1,3200000.0,3200000.0
3,2022-02-12,NN-CARSPA,TOYOTA,04003-45450,Cụm ABS,1,NaN,27000000.0
4,2022-02-12,ANHCUONG,MAZDA,B4Y0-26-48ZB,Má phanh sau,1,1400000.0,1400000.0
...,...,...,...,...,...,...,...,...
2728,2022-08-30,NN-CHPT-TAY,KYOSAN,TP-656,Bơm xăng cơ,1,NaN,580000.0
2729,2022-08-30,HUUHANH,TOYOTA,13506-13040,Xích cam (2 mắt vàng),1,690000.0,750000.0
2730,2022-08-30,HUUHANH,RBI,T1024R,Chân máy Toyota Zace RH,1,293000.0,320000.0
2731,2022-08-30,KL-CCMFAST,TOYOTA,42431-60311,Đĩa phanh sau $31 - 6 lỗ,1,2850000.0,2850000.0


In [8]:
# TEST CÁC MÃ HÀNG 

# data_muahang_latest.query("ID=='48067-29215-MIT'")
# data_muahang_min.query("ID=='48760-33130'")
# data_muahang.query("ID=='48760-33130'").sort_values(by="Giá nhập",ascending=True)
data_banhang.query("ID=='16604-31020'").sort_values(by="Ngày bán",ascending=False)
data_banhang_latest.query("ID=='16604-31020'")

,Ngày bán gần đây,ID Khách hàng,Hãng,ID,Tên hàng,Tổng bán,Giá bán,Giá bán gần đây
1202,2022-10-05,KIET-CHUBA,TOYOTA,16604-31020,Bi tỳ máy phát 1GR/2GR (2),2,650000.0,630000.0


In [9]:
data_muahang.query("ID=='48760-33130'").sort_values(by="Giá nhập",ascending=True)

,Ngày nhập,Nhà cung cấp,ID,Tổng nhập,Giá nhập
11399,2022-03-29,NK-TS,48760-33130,2,937060.0
2790,2020-07-14,HTAUTOHN,48760-33130,3,965000.0
3760,2020-09-15,HTAUTOHN,48760-33130,5,1007000.0
41,2020-01-04,HTAUTOHN,48760-33130,1,1066000.0
5845,2020-12-28,HTAUTOHN,48760-33130,3,1100000.0
5858,2020-12-28,HTAUTOHN,48760-33130,3,1100000.0
11287,2022-03-28,CHPT-MINHANH,48760-33130,1,1140000.0
11556,2022-04-02,CHPT-MINHANH,48760-33130,1,1140000.0


In [10]:
# Dữ liệu Nhập Bán Tồn
data_nhap_ban_ton=data_nhapxuatton.merge(
            data_giaban,on="ID",how="left").merge(
                                            data_banhang_latest[["ID","Ngày bán gần đây","Giá bán gần đây"]],on="ID",how="left").sort_values(
                                                by="Giá bán gần đây",ascending=True
                                            )
data_nhap_ban_ton_filter=data_nhap_ban_ton[["ID","Giá vốn","Giá bán","Giá bán gần đây","Tổng nhập","Tổng xuất","Tổng tồn"]].reset_index(drop=True)
data_nhap_ban_ton_filter

# Gộp dữ liệu Mua hàng Latest & MIn, Bán hàng Latest và Giá bán, Tổng Nhập Tổng Xuất Tổng Tồn
data_muahang_latest_min=data_muahang_latest.merge(data_muahang_min,on="ID",how="left")
data_NCC=data_muahang_latest_min.merge(data_nhap_ban_ton_filter,on="ID").sort_values(
                                        by="Tổng tồn",ascending=True).reset_index(drop=True)

In [11]:
data_NCC

,Ngày nhập gần đây,Nhà cung cấp gần đây,ID,Tổng nhập gần đây,Giá nhập gần đây,Ngày nhập MIN,Nhà cung cấp MIN,Tổng nhập MIN,Giá nhập MIN,Giá vốn,Giá bán,Giá bán gần đây,Tổng nhập,Tổng xuất,Tổng tồn
0,2021-07-09,NK-CHI,88440-0K011,100,524844.00,2020-03-24,HTAUTOHN,3,313000.00,622217.0,700000.0,650000.0,4,91,-2
1,2022-08-13,NCC-QUANGNINH,47201-0K040,17,1170000.00,2022-08-13,NCC-QUANGNINH,17,1170000.00,1256186.0,1400000.0,1600000.0,101,103,-1
2,2022-05-21,NCC-NAMCHAU,06E-121-065N,1,800000.00,2022-05-21,NCC-NAMCHAU,1,800000.00,861248.0,NaN,1400000.0,1,1,0
3,2022-05-20,CHPT-MINHANH,44310-06130,1,5000000.00,2022-05-13,NK-CHI,5,3604111.00,4327369.0,6050000.0,6000000.0,14,14,0
4,2022-05-20,CHPT-MINHANH,90919-01253,17,215000.00,2020-09-14,HTAUTOHN,50,166000.00,225928.0,250000.0,250000.0,21,32,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2831,2022-06-03,NK-IDAS,JK272000-6930,3550,142515.75,2022-01-26,HTAUTOHN,200,139000.00,151677.0,170000.0,170000.0,9136,6980,2156
2832,2022-09-16,NK-TS,SR-3880,2670,155526.52,2022-08-01,NN-QUYEN,60,0.00,151174.0,230000.0,230000.0,3922,2600,2162
2833,2022-05-10,NK-IDAM,90919-T1004,15000,19156.40,2021-05-24,NK-IDAM,7000,18759.60,20701.0,28000.0,25000.0,34200,38961,2827
2834,2022-09-27,NK-IDAS,08889-80280,1200,139138.00,2022-04-05,NK-IDAS,500,135416.40,151673.0,220000.0,200000.0,10084,7182,3620


In [12]:
# data_NCC.query("ID=='48760-33130'")

,Ngày nhập gần đây,Nhà cung cấp gần đây,ID,Tổng nhập gần đây,Giá nhập gần đây,Ngày nhập MIN,Nhà cung cấp MIN,Tổng nhập MIN,Giá nhập MIN,Giá vốn,Giá bán,Giá bán gần đây,Tổng nhập,Tổng xuất,Tổng tồn
502,2022-04-02,CHPT-MINHANH,48760-33130,1,1140000.0,2022-03-29,NK-TS,2,937060.0,1168839.0,1450000.0,1400000.0,5,5,0


In [ ]:
# count_danh_muc=data_danhmuchanghoa.groupby(["ID Nhóm hàng"]).count()

In [25]:
data_NCC_2022=data_NCC[(data_NCC["Ngày nhập gần đây"].dt.strftime('%Y')=='2022') & (data_NCC["Ngày nhập MIN"].dt.strftime('%Y')=='2022') ]
data_NCC_NK=data_NCC.query('(`Nhà cung cấp gần đây`.str.contains("NK-")) & (`Nhà cung cấp MIN`.str.contains("NK-"))', engine='python').reset_index(drop=True)


In [26]:
# EXPORT FILE DATA
with pd.ExcelWriter('../VACP/Python NCC Min/NCC và NK nho nhat nam 2020-2022 (Python)/Report_NCC.xlsx') as writer:
    data_NCC.to_excel(writer,sheet_name="Đầy đủ")
    data_NCC_2022.to_excel(writer,sheet_name="Năm 2022")
    data_NCC_NK.to_excel(writer,sheet_name="Chỉ NK")
    # data_report.to_excel(writer,sheet_name="Tổng hợp")
    # data_danhmuchanghoa.to_excel(writer,sheet_name="Danh mục hàng")
    # count_danh_muc.to_excel(writer,sheet_name="đến số lượng")
